In [22]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/Othercomputers/Mac/dsan6600'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Mac/dsan6600


In [26]:
import torchvision
import torch
from torchvision.io.image import read_image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

In [13]:
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights = weights, box_score_thresh = 0.8)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:01<00:00, 104MB/s] 


In [20]:
# get the number of input heads from the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

In [27]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

# define transformations
data_transform = transforms.Compose([transforms.ToTensor()])

# create datasets
train_dataset = CocoDetection('Project.v1i.coco/train',
                              'Project.v1i.coco/train/_annotations.coco.json',
                              transform = data_transform)
val_dataset = CocoDetection('Project.v1i.coco/valid',
                              'Project.v1i.coco/valid/_annotations.coco.json',
                              transform = data_transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
import torch.optim as optim

# set up optimizer, loss, and learning rate scheduler
optimizer = optim.SGD(model.parameters(), lr = 0.005, momentum = 0.9, weight_decay = 0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

# training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training loop
    for images, targets in train_loader:
        outputs = model(images, targets)
        print(outputs.keys())
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        for images, targets in val_loader:
            outputs = model(images)
            # Compute evaluation metrics

    # Update learning rate
    lr_scheduler.step()

    # Save model checkpoint
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')

In [40]:
sample = train_dataset[0]  # Get the first sample
image, targets = sample
targets[0]

{'id': 0,
 'image_id': 0,
 'category_id': 1,
 'bbox': [169, 281, 11, 18],
 'area': 198,
 'segmentation': [],
 'iscrowd': 0}

In [50]:
import torch

def convert_targets(targets):
    boxes = torch.tensor([t['bbox'] for t in targets], dtype=torch.float32)
    labels = torch.tensor([t['category_id'] for t in targets], dtype=torch.int64)
    image_id = torch.tensor([t['image_id'] for t in targets])
    area = torch.tensor([t['area'] for t in targets], dtype=torch.float32)
    iscrowd = torch.tensor([t['iscrowd'] for t in targets], dtype=torch.int64)

    converted_targets = {
        'boxes': boxes,
        'labels': labels,
        'image_id': image_id,
        'area': area,
        'iscrowd': iscrowd
    }

    return converted_targets

# Example usage
targets_list = [
    {'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [169, 281, 11, 18], 'area': 198, 'segmentation': [], 'iscrowd': 0},
    {'id': 1, 'image_id': 0, 'category_id': 2, 'bbox': [200, 300, 20, 30], 'area': 600, 'segmentation': [], 'iscrowd': 0},
    # Add more targets dictionaries here
]

converted_targets = convert_targets(targets)

In [53]:
for images, targets in train_loader:
    outputs = model(images, convert_targets(targets))
    print(outputs.keys())

ValueError: only one element tensors can be converted to Python scalars

In [57]:
for images, targets in val_loader:
  print(targets)

RuntimeError: each element in list of batch should be of equal size